In [1]:
import sys
import torch
import os
from os import makedirs
import numpy as np
import open3d as o3d
from random import randint
from scipy.spatial.transform import Rotation
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2

np.set_printoptions(suppress=True)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def getWorld2View2(R, t, translate=np.array([.0, .0, .0]), scale=1.0):
    Rt = np.zeros((4, 4))
    Rt[:3, :3] = R.transpose()
    Rt[:3, 3] = t
    Rt[3, 3] = 1.0

    C2W = np.linalg.inv(Rt)
    cam_center = C2W[:3, 3]
    cam_center = (cam_center + translate) * scale
    C2W[:3, 3] = cam_center
    Rt = np.linalg.inv(C2W)
    return np.float32(Rt)

def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])

In [3]:
# Camera Intrinsics
rgb_camera_params = {}
depth_camera_params = {}
relative_positions = {}

def readRGBDConfig(config_file):
    # Define dictionaries to hold camera parameters
    # rgb_camera_params = {}
    # depth_camera_params = {}
    # relative_positions = {}
    
    with open(config_file, 'r') as file:
        data = file.read().split('\n\n')
    
        # Read RGB camera parameters
        rgb_data = data[0].split('\n')
        for line in rgb_data[1:4]:
            key, value = line.split('=')
            if ',' in value:
                value = tuple(map(float, value.split(',')))
            else:
                value = tuple(map(int, value.split('x')))
            rgb_camera_params[key] = value

        vFOV, hFOV = rgb_data[4].split(',')
        key, value = vFOV.split('=')
        rgb_camera_params[key] = float(value.strip('°'))
        key, value = hFOV.split('=')
        rgb_camera_params[key.strip(' ')] = float(value.strip('°'))

        # Read Depth camera parameters
        depth_data = data[1].split('\n')
        for line in depth_data[1:4]:
            key, value = line.split('=')
            if ',' in value:
                value = tuple(map(float, value.split(',')))
            else:
                value = tuple(map(int, value.split('x')))
            depth_camera_params[key] = value

        vFOV, hFOV = depth_data[4].split(',')
        key, value = vFOV.split('=')
        depth_camera_params[key] = float(value.strip('°'))
        key, value = hFOV.split('=')
        depth_camera_params[key.strip(' ')] = float(value.strip('°'))

    
        # Read relative positions of camera components
#         rel_pos_data = data[2].split('\n')
#         for line in rel_pos_data[1:]:
#             key, value = line.split(': ')
#             value = tuple(map(float, value.strip('(').strip(')').split(',')))
#             relative_positions[key] = value

        # return rgb_camera_params, depth_camera_params, relative_positions
    
    # Access the loaded camera parameters
    print("RGB Camera Parameters:")
    print(rgb_camera_params)
    
    print("\nDepth Camera Parameters:")
    print(depth_camera_params)
    
    print("\nRelative Positions of Camera Components:")
    print(relative_positions)

#config_file = "/home/presnov/Desktop/data/Workspace/PolyRep/synthData/SyntheticV2/configuration.txt"#tableScene-v12/config/configuration.txt"#"G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\config\\configuration.txt"
# config_file = "/home/presnov/Desktop/data/Workspace/PolyRep/tableScene-v12/config/configuration.txt"
config_file = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\SyntheticV1\\config\\configuration.txt"

readRGBDConfig(config_file)

RGB Camera Parameters:
{'resolution': (1920, 1080), 'cx,cy,fx,fy': (959.5, 539.5, 960.0, 960.0), 'k1,k2,k3,p1,p2': (0.0, 0.0, 0.0, 0.0, 0.0), 'vFov': 58.7155, 'hFov': 90.0}

Depth Camera Parameters:
{'resolution': (512, 424), 'cx,cy,fx,fy': (255.5, 211.5, 256.0, 256.0), 'k1,k2,k3,p1,p2': (0.0, 0.0, 0.0, 0.0, 0.0), 'vFov': 79.2579, 'hFov': 89.9999}

Relative Positions of Camera Components:
{}


In [4]:
# Camera Extrinsics
#directory = "/home/presnov/Desktop/data/Workspace/PolyRep/synthData/SyntheticV2/trajectory"#tableScene-v12/config"#"G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\config"
# directory = "/home/presnov/Desktop/data/Workspace/PolyRep/tableScene-v12/config"
directory = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\SyntheticV1\\config"


config_files = os.listdir(directory)
config_files.sort()

extrinsics = {}

# config_files[:5]
# for config_index in tqdm(range(5)):
for config_index in tqdm(range(len(config_files))):
    file = config_files[config_index]
    if file.startswith("campose-rgb-"):#campose-tof-"):#
        frame_id = file.split('-')[2].split('.')[0]
        config_file = os.path.join(directory, file)
        # print(config_file)
        
        with open(config_file, 'r') as file:
            lines = file.readlines()

            # Extracting position
            position_str = lines[0].replace('position=', '').split('\n')[0]
            position = np.array([float(i) for i in position_str.strip('()').split(',')])

            # Extracting rotation as a quaternion
            rotation_str = lines[1].replace('rotation_as_quaternion=', '').split('\n')[0]
            rotation = np.array([float(i) for i in rotation_str.strip('()').split(',')])

            # Extracting the 4x4 pose matrix
            pose_str = lines[3:]
            pose = np.array([[float(i) for i in row.strip('(').split(')')[0].split(',')] for row in pose_str if row != ''])    

            # print('Position:', position)
            # print('Rotation:',rotation)
            # print('Pose:',pose)
            extrinsics[frame_id] = pose#np.linalg.inv(pose)
            #print(extrinsics[frame_id])

100%|██████████| 601/601 [00:00<00:00, 14690.44it/s]


In [5]:
print(len(extrinsics))


300


In [6]:
# Directory where your images are stored
#directory = "/home/presnov/Desktop/data/Workspace/PolyRep/synthData/SyntheticV2/gt_depth/"#"#tableScene-v12/rgb"#tof-depth"#"G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\rgb"
# directory = "/home/presnov/Desktop/data/Workspace/PolyRep/tableScene-v12/rgb"#tof-depth"
# ply_path = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\ply"
directory = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\SyntheticV1\\rgb"
ply_path = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\ply"

# Get the list of files in the directory
files = os.listdir(directory)

# Sort the files to process depth and color images together
files.sort()

# Initialize an empty point cloud
point_cloud = o3d.geometry.PointCloud()

start_index = 0
# ply_files = os.listdir(ply_path)
# if len(ply_files) > 0:
#     start_index = int(ply_files[-1].split('.')[0])

frame_step = 10
# Loop through each pair of depth and color images
# for i in tqdm(range(1)):

depth_files = [file for file in files if file.startswith("gt-rgb-depth-") ]#gt-tof-depth-") ]#
depth_files = depth_files[50:150]

print("Number of files:", len(depth_files))

Number of files: 100


In [19]:
import copy

# refPose = extrinsics['0000']
# refPose = linalg.inv(refPose)

# import matplotlib.pyplot as plt
# %matplotlib

# fig = plt.figure()
# axs = fig.add_subplot(projection='3d')

point_cloud = o3d.geometry.PointCloud()

# Using Numpy to create point clouds
#for i in tqdm(range(start_index, len(depth_files),frame_step)):
# for i in range(start_index, len(files),frame_step):#
count = 0
pcds = []
points = []
axes = []
# ax = o3d.geometry.TriangleMesh.create_coordinate_frame(size=15)
# axes.append(ax)

colmap = np.array([[1,0,0], [0,1,0], [0,0,1]])

#for i in tqdm(range(0,99,33)):
for i in tqdm(range(start_index, len(depth_files),frame_step)):
    depth_file_name = depth_files[i]
    depth_file = os.path.join(directory, depth_file_name)
    
    # Get the corresponding color image
    color_file = os.path.join(directory, "rgb-" + depth_file_name[-8:])  # Assuming both files have corresponding indices
    
    frame_id = depth_file_name.split('-')[3].split('.')[0]

    # Read the depth and color images
    color_image = cv2.imread(color_file)
    depth_image = cv2.imread(depth_file, cv2.IMREAD_ANYDEPTH)# cv2.IMREAD_UNCHANGED)#,flags=(cv2.IMREAD_GRAYSCALE | cv2.IMREAD_ANYDEPTH)).astype(np.float32)#, cv2.IMREAD_UNCHANGED)
    
    

    # Convert images to numpy arrays
    depth_array = np.asarray(depth_image)
    color_array = np.asarray(color_image)
    
    #print("original depth: ", depth_array[600:610, 500:510])

    width = depth_array.shape[1]
    height = depth_array.shape[0]

    # Intrinsic parameters of the camera (you may need to adjust these values)
    cx = rgb_camera_params['cx,cy,fx,fy'][0]
    cy = rgb_camera_params['cx,cy,fx,fy'][1]
    fx = rgb_camera_params['cx,cy,fx,fy'][2] 
    fy = rgb_camera_params['cx,cy,fx,fy'][3]

#     cx = depth_camera_params['cx,cy,fx,fy'][0]
#     cy = depth_camera_params['cx,cy,fx,fy'][1]
#     fx = depth_camera_params['cx,cy,fx,fy'][2] 
#     fy = depth_camera_params['cx,cy,fx,fy'][3]


    # print("I:", intrinsic.intrinsic_matrix)
    # print("P:", extrinsics[frame_id])

    assert frame_id in extrinsics, "No pose found for frame " + frame_id

    pose = extrinsics[frame_id]

    print("Frame:", frame_id)
    print("Pose:", pose)

    # Extraction of rotation matrix (3x3 sub-matrix)
    rotation_matrix = pose[:3, :3]

    #print(rotation_matrix)

    # Extraction of translation vector (last column)
    translation_vector = pose[:3, 3]
    
    #!!!!!!!!!!
    #pose[0, 3]= pose[0, 3] - 0.03
    #!!!!!!!!

    print("translation ", translation_vector)

    # Convert depth to 3D coordinates in camera coordinates
#     u, v = np.meshgrid(np.arange(width), np.arange(height))
    u, v = np.meshgrid(np.arange(width), np.arange(height-1, -1, -1))
    u = u.flatten()
    v = v.flatten()

    # Convert depth to float32 for accurate calculations
    depth_image = depth_image.astype(np.float32)
    #depth_image = depth_image[:,:,0].astype(np.float32)# / 255.
    
    #print("converted depth: ", depth_array[600:610, 500:510])
    depth_scale = 1.0 / 10000.0#1000.0 # The depth scale of the Intel RealSense D435i is 0.001
    depth = depth_image.flatten() * depth_scale
    
#     cx = cx + 0.5
#     cy = cy - 0.5
    X = ((u - cx) * depth / fx) 
    Y = ((v - cy) * depth / fy)
    Z = -depth

    
#     X = (u - cx) / fx 
#     Y = (v - cy) / fy   
#     prLength = np.sqrt(np.power(X, 2) + np.power(Y,2) + 1)    
#     Z = depth / prLength
#     X = Z * X
#     Y = Z * Y
    
     
#     print("u ", u[900], " cx ", cx, " u shifted ", u[900] - cx, " fx ", fx, " X 2d ",\
#           (u[900] - cx) / fx,\
#          " depth ", depth[1920*540+900], " X ", (u[900] - cx) * depth[1920*540+900] / fx)
    #print("x: ", X[:5])

    # Stack the 3D coordinates
    points_camera = np.vstack((X, Y, Z, np.ones_like(X)))

    if False:
        # Extract the transformed X, Y, Z coordinates
        X_local = points_camera[0, :]
        Y_local = points_camera[1, :]
        Z_local = points_camera[2, :]
        # Stack the global coordinates
        point_cloud_local = np.vstack((X_local, Y_local, Z_local)).T
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_cloud_local)
        pcd.transform([[1, 0, 0, 0],[0, -1, 0, 0],[0, 0, -1, 0],[0, 0, 0, 1]])
        point_cloud += pcd

        
    #*******
    ax = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2)
    pcds.append(copy.deepcopy(ax).transform(pose))
    ax = None
    #*********
    
    
    # Transform to global coordinates
    #print(pose)
    #points_global = np.dot(np.dot(refPose,pose), points_camera)
    
    points_global = np.dot(pose, points_camera)  
    
    #points_global = copy.deepcopy(points_camera)#np.dot(extrinsics['0000'], points_camera) 
    #print("shape ", np.shape(points_global))
    

    
    #points_global = points_camera
    # Extract the transformed X, Y, Z coordinates
    X_global = points_global[0, :]
    Y_global = points_global[1, :]
    Z_global = points_global[2, :]
    # Stack the global coordinates
    point_cloud_global = np.vstack((X_global, Y_global, Z_global)).T
    
#     startInd = 150000
#     endInd = 155000
#     if count==0:
#         img = axs.scatter(point_cloud_global[startInd:endInd,0], point_cloud_global[startInd:endInd,1],\
#                           point_cloud_global[startInd:endInd,2], c='r')
#     else:
#         img = axs.scatter(point_cloud_global[startInd:endInd,0], point_cloud_global[startInd:endInd,1],\
#                           point_cloud_global[startInd:endInd,2], c='b')
    
    #points.append(point_cloud_global)

    # Create an Open3D PointCloud
    pcd = o3d.geometry.PointCloud()
    # Set the points in the PointCloud
    pcd.points = o3d.utility.Vector3dVector(point_cloud_global)

    # Flip the point cloud
    #pcd.transform([[1, 0, 0, 0],[0, -1, 0, 0],[0, 0, -1, 0],[0, 0, 0, 1]])

    rgb_values = color_array.reshape((-1, 3)) / 255.0
    pcd.colors = o3d.utility.Vector3dVector(rgb_values)
    
#     rgb_values = np.ones((np.shape(point_cloud_global)[0], 3))
#     rgb_values = rgb_values * colmap[count%3]
#     pcd.colors = o3d.utility.Vector3dVector(rgb_values)
    
    # Merge current point cloud with the overall point cloud
 
    #point_cloud += pcd
    pcds.append(pcd)
    
   
    

    # Clear memory
    depth_image = None
    color_image = None
    depth_array = None
    color_array = None
    pcd = None
    count = count + 1


# Create an Open3D mesh representing coordinate axes
#axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=[0, 0, 0])

# Visualize the final point cloud
o3d.visualization.draw_geometries(pcds)#[point_cloud, pcd, axes])

#plt.show()

  0%|          | 0/10 [00:00<?, ?it/s]

Frame: 0050
Pose: [[-0.996195    0.00000004 -0.0871558   0.483333  ]
 [-0.0435779   0.866025    0.498097    0.916667  ]
 [ 0.0754792   0.5        -0.86273     2.44434   ]
 [ 0.          0.          0.          1.        ]]
translation  [0.483333 0.916667 2.44434 ]


 10%|█         | 1/10 [00:02<00:21,  2.34s/it]

Frame: 0060
Pose: [[-0.994522    0.00000004 -0.104529    0.48      ]
 [-0.0522642   0.866025    0.497261    0.9       ]
 [ 0.0905244   0.5        -0.861281    2.47321   ]
 [ 0.          0.          0.          1.        ]]
translation  [0.48    0.9     2.47321]


 20%|██        | 2/10 [00:04<00:18,  2.31s/it]

...

 80%|████████  | 8/10 [00:18<00:04,  2.28s/it]

Frame: 0130
Pose: [[-0.97437     0.00000004 -0.224951    0.449968  ]
 [-0.112476    0.866025    0.487185    0.804984  ]
 [ 0.194813    0.5        -0.843829    2.68778   ]
 [ 0.          0.          0.          1.        ]]
translation  [0.449968 0.804984 2.68778 ]


 90%|█████████ | 9/10 [00:20<00:02,  2.31s/it]

Frame: 0140
Pose: [[-0.970296    0.00000005 -0.241922    0.428333  ]
 [-0.120961    0.866025    0.485148    0.804167  ]
 [ 0.209511    0.5        -0.840301    2.7258    ]
 [ 0.          0.          0.          1.        ]]
translation  [0.428333 0.804167 2.7258  ]


100%|██████████| 10/10 [00:23<00:00,  2.31s/it]


In [9]:
from scipy.spatial.transform import Rotation as R

In [10]:
r = R.from_quat([-0.000451713,0.965924,0.258819,-0.0016859])
r.as_matrix()

array([[-0.99999391,  0.00000005, -0.00349073],
       [-0.00174533,  0.86602497,  0.4999977 ],
       [ 0.00302308,  0.50000074, -0.8660197 ]])

In [11]:
r = R.from_quat([-0.000451713,0.965924,0.258819,-0.0016859])
r.as_matrix()

array([[-0.99999391,  0.00000005, -0.00349073],
       [-0.00174533,  0.86602497,  0.4999977 ],
       [ 0.00302308,  0.50000074, -0.8660197 ]])

In [12]:
from numpy import linalg
v = np.array([0.965924,0.258819,-0.0016859])
linalg.norm(v)

0.9999996453978421

In [13]:
#linalg.inv(extrinsics[0])
extrinsics

{'0000': array([[-1.        ,  0.00000004, -0.00000008,  0.5       ],
        [-0.        ,  0.866025  ,  0.5       ,  1.        ],
        [ 0.00000009,  0.5       , -0.866025  ,  2.3       ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 '0001': array([[-0.999999  ,  0.00000004, -0.0017454 ,  0.499667  ],
        [-0.00087266,  0.866025  ,  0.499999  ,  0.998333  ],
        [ 0.00151159,  0.5       , -0.866024  ,  2.30289   ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 '0002': array([[-0.999994  ,  0.00000004, -0.00349073,  0.499333  ],
        [-0.00174533,  0.866025  ,  0.499997  ,  0.996667  ],
        [ 0.00302308,  0.5       , -0.86602   ,  2.30577   ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 '0003': array([[-0.999986  ,  0.00000004, -0.00523604,  0.499     ],
        [-0.00261798,  0.866025  ,  0.499993  ,  0.995     ],
        [ 0.00453457,  0.5       , -0.866014  ,  2.30866   ],
        [ 0.        ,  0.       

In [14]:
depth[15000:15050]

array([1.5260999, 1.5249   , 1.5236   , 1.5223999, 1.5210999, 1.5199   ,
       1.5186   , 1.5173999, 1.5160999, 1.5149   , 1.5137   , 1.5123999,
       1.5112   , 1.51     , 1.5087   , 1.5074999, 1.5063   , 1.5051999,
       1.5041   , 1.5029   , 1.5018   , 1.5007   , 1.4995999, 1.4985   ,
       1.4973999, 1.4963   , 1.4951999, 1.4941   , 1.4929999, 1.4919   ,
       1.4907999, 1.4897   , 1.4886   , 1.4875   , 1.4864   , 1.4853   ,
       1.4842   , 1.4830999, 1.482    , 1.481    , 1.4799   , 1.4787999,
       1.4777   , 1.4767   , 1.4757   , 1.4747   , 1.4735999, 1.4726   ,
       1.4715999, 1.4706   ], dtype=float32)

In [15]:
np.amax(depth)#_image2)

5.2841

In [16]:
#depth2 = depth_image.flatten()
# depth_image[99:100, 50:100]#/255

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# depth_image2[99:100, 50:100]
#depth_file_name

array([[25658., 25685., 25713., 25740., 25767., 25795., 25822., 25850.,
        25878., 25906., 25933., 25961., 25989., 26017., 26045., 26073.,
        26101., 26130., 26158., 26186., 26215., 26243., 26272., 26300.,
        26329., 26357., 26386., 26415., 26444., 26473., 26502., 26531.,
        26560., 26589., 26619., 26648., 26677., 26707., 26736., 26766.,
        26796., 26825., 26855., 26885., 26915., 26945., 26975., 27005.,
        27035., 27066.]], dtype=float32)

In [17]:
depth_file_name = depth_files[20]
depth_file = os.path.join(directory, depth_file_name)


# Read the depth and color images
#color_image = cv2.imread(color_file)
depth_image2 = cv2.imread(depth_file, flags=(cv2.IMREAD_GRAYSCALE | cv2.IMREAD_ANYDEPTH)).astype(np.float32)#cv2.IMREAD_UNCHANGED)#GRAYSCALE)#

In [18]:
print(type(depth_image))
print(depth_image.shape)
cv2.imshow('depth', depth_image)

<class 'NoneType'>


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
#pc1 = copy.deepcopy(point_cloud_global)
pc2 = copy.deepcopy(point_cloud_global)

In [ ]:
allpoints = np.vstack((pc1[:500], pc2[:500]))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib

fig = plt.figure()
axs = fig.add_subplot(projection='3d')
img = axs.scatter(allpoints[:,0], allpoints[:,1], allpoints[:,2])


plt.show()

Using matplotlib backend: Qt5Agg


In [ ]:
np.shape(point_cloud_global)

(217088, 3)

In [ ]:
rot = qvec2rotmat([1.13133e-08,0.965926,0.258819,-4.2222e-08])
rot2 = Rotation.from_quat([1.13133e-08,0.965926,0.258819,-4.2222e-08]).as_matrix()

print(rot)
print('')
print(rot2)

[[ 0.86602545  0.5        -0.00000008]
 [ 0.5        -0.86602607 -0.00000004]
 [-0.00000009 -0.         -1.00000062]]

[[-1.          0.00000004 -0.00000008]
 [-0.          0.86602549  0.49999985]
 [ 0.00000009  0.49999985 -0.86602549]]


In [ ]:
# Visualize the final point cloud
ply_file_path = os.path.join(ply_path, '0290.ply')
# ply_file_path = os.path.join(ply_path, 'Global.ply')

# 4X4 transformation matrix
transformation_matrix =  [[1, 0, 0, 0.5],
                            [0, 1, 0, 0],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]]


ply_files = os.listdir(ply_path)
pcd_list = []
for file in ply_files[:1]:
    if file.endswith(".ply"):
        pcd_file = os.path.join(ply_path, file)
        pcd = o3d.io.read_point_cloud(pcd_file)
        pcd.transform(transformation_matrix)
        pcd_list.append(pcd)

pcd_list.append(o3d.io.read_point_cloud(ply_file_path))
# o3d.visualization.draw_geometries(pcd_list)


[Open3D WARNING] Read PLY failed: unable to open file: G:\Universitat Siegen\SA\P-GPU\Code\gaussian-splatting\data\RGBD_Data\ply\0290.ply


In [ ]:
#************
import numpy as np

In [ ]:
Kd = np.array([[256, 0, 959.5],\
                  [0, 256, 539.5],\
                  [0,0,1]])

Krgb = np.array([[960, 0, 959.5],\
                  [0, 960, 539.5],\
                  [0,0,1]])

u = np.array([100, 100, 1])

T = np.eye(4)
T[2,3] = 0.5

print(T)

s = 1#000
d1 = 1 * s
d2 = 0.5 * s

p1 = np.array([1,1,1])
p2 = np.array([1,1,0.5])

[[1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.  1.  0.5]
 [0.  0.  0.  1. ]]


In [ ]:
u1_d = Kd.dot(p1)
print(u1_d)

u1_rgb = Krgb.dot(p1)
print(u1_rgb)


[1.2155e+03 7.9550e+02 1.0000e+00]
[1.9195e+03 1.4995e+03 1.0000e+00]


In [ ]:
u2_d = Kd.dot(p2)
print(u2_d)
u2_rgb = Krgb.dot(p2)
print(u2_rgb)

#np.linalg.inv(Kd)

#np.linalg.inv(T).dot(np.concatenate((p2,[1])))

[7.3575e+02 5.2575e+02 5.0000e-01]
[1.43975e+03 1.22975e+03 5.00000e-01]


In [ ]:
# T.dot(np.concatenate((np.linalg.inv(Kd).dot(u2_d), [1])))
print(T.dot(np.concatenate((np.linalg.inv(Kd).dot(u2_d), [1]))))

print(T.dot(np.concatenate((np.linalg.inv(Krgb).dot(u2_d), [1]))))

[1. 1. 1. 1.]
[0.26666667 0.26666667 1.         1.        ]


In [ ]:
np.linalg.inv(Kd).dot(u2_d)

array([1. , 1. , 0.5])

In [ ]:
p1_d = d1*Kd_inv.dot(u)
print("point from depth", p1_d)

p1_rgb = d1*Krgb_inv.dot(u)
print("point from rgb", p1_rgb)

p2_d = d2*Kd_inv.dot(u)
print("point from depth", p2_d, T.dot(np.concatenate((p2_d, [1]))))

p2_rgb = d2*Krgb_inv.dot(u)
print("point from rgb", p2_rgb, T.dot(np.concatenate((p2_rgb, [1]))))

point from depth [-959.109375 -539.109375    1.      ]
point from rgb [-959.39583333 -539.39583333    1.        ]
point from depth [-479.5546875 -269.5546875    0.5      ] [-479.5546875 -269.5546875    1.           1.       ]
point from rgb [-479.69791667 -269.69791667    0.5       ] [-479.69791667 -269.69791667    1.            1.        ]


In [ ]:
Kd_inv[0].dot(u)

-959.109375